<a href="https://colab.research.google.com/github/sahilpate3345/deep-learning/blob/main/Text_summerizer_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q


In [ ]:
!pip install --upgrade accelerate
!pip install transformers accelerate


In [ ]:
from transformers import pipeline,set_seed
from datasets import load_dataset,load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch
nltk.download("punkt")

In [ ]:
device="cuida" if torch.cuda.is_available() else "cpu"
device


In [ ]:
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer

In [ ]:
model_ckpt="google/pegasus-cnn_dailymail"
tokenizer=AutoTokenizer.from_pretrained(model_ckpt)#tokenizer is used to convert in vector

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_ckpt = "google/pegasus-xsum"  # example checkpoint
device = "cuda" if torch.cuda.is_available() else "cpu"

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)


In [ ]:
from datasets import load_dataset

dataset_samsum = load_dataset("knkarthick/samsum")
print(dataset_samsum)


In [ ]:
dataset_samsum

In [ ]:
dataset_samsum["train"]["dialogue"][1]

In [ ]:
dataset_samsum["train"][1]["summary"]

In [ ]:
split_lengths=[len(dataset_samsum[split])for split in dataset_samsum]
print(f"split length:{split_lengths}")
print(f"features:{dataset_samsum['train'].column_names}")
print("\nDialogue")
print(dataset_samsum["test"][1]["dialogue"])
print("\n summary")
print(dataset_samsum["test"][1]["summary"])





In [ ]:
def convert_example_to_feature(example_batch):
    # Tokenize the dialogue (input)
    inputs = tokenizer(example_batch['dialogue'],
                       max_length=1024,
                       truncation=True,
                       padding="max_length")

    # Tokenize the summary (target)
    targets = tokenizer(example_batch['summary'],
                        max_length=128,
                        truncation=True,
                        padding="max_length")

    # Return tokenized inputs and labels
    inputs["labels"] = targets["input_ids"]
    return inputs

In [ ]:
dataset_samsum_pt = dataset_samsum.map(convert_example_to_feature, batched=True)

In [ ]:
dataset_samsum_pt["train"]

In [ ]:
def convert_example_to_feature(example_batch):
    # Tokenize the dialogue (input)
    inputs = tokenizer(
        example_batch['dialogue'],
        max_length=1024,
        truncation=True,
        padding="max_length",
        return_attention_mask=True
    )

    # Tokenize the summary (target)
    targets = tokenizer(
        example_batch['summary'],
        max_length=128,
        truncation=True,
        padding="max_length",
        return_attention_mask=True
    )

    # Return tokenized inputs and labels
    inputs["labels"] = targets["input_ids"]
    inputs["decoder_attention_mask"] = targets["attention_mask"]

    return inputs


In [ ]:
dataset_samsum_pt = dataset_samsum.map(convert_example_to_feature, batched=True)


In [ ]:
dataset_samsum_pt["train"][0].keys()


In [ ]:
dataset_samsum_pt["train"]["attention_mask"][1]

In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)
from datasets import load_dataset


In [ ]:
model_ckpt = "google/pegasus-xsum"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt)

# Use GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


In [ ]:
dataset_samsum = load_dataset("knkarthick/samsum")  # or your preferred namespace

def convert_example_to_feature(example_batch):
    inputs = tokenizer(
        example_batch['dialogue'],
        max_length=1024,
        truncation=True,
        padding="max_length",
        return_attention_mask=True
    )
    targets = tokenizer(
        example_batch['summary'],
        max_length=128,
        truncation=True,
        padding="max_length",
        return_attention_mask=True
    )
    inputs["labels"] = targets["input_ids"]
    inputs["decoder_attention_mask"] = targets["attention_mask"]
    return inputs

dataset_samsum_pt = dataset_samsum.map(convert_example_to_feature, batched=True)


In [ ]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    padding=True,
    return_tensors="pt"
)


In [ ]:
pip install --upgrade transformers


In [ ]:
import transformers
print(transformers.__version__)  # should be >=4.5.0


In [ ]:
from transformers import Seq2SeqTrainingArguments
import torch  # for torch.cuda.is_available()

training_args = Seq2SeqTrainingArguments(
    output_dir="./pegasus_samsum",
    do_train=True,
    do_eval=True,
    logging_steps=100,
    save_steps=500,
    save_total_limit=2,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    learning_rate=5e-5,
    weight_decay=0.01,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),  # use mixed precision if GPU
    logging_dir="./logs",
)


In [ ]:
from datasets import load_dataset

dataset_samsum_pt = load_dataset("knkarthick/samsum")


In [ ]:
from transformers import PegasusTokenizer

model_name = "google/pegasus-large"
tokenizer = PegasusTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    inputs = examples["dialogue"]
    targets = examples["summary"]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    labels = tokenizer(targets, max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset_samsum_pt.map(preprocess_function, batched=True)


In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)


In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./pegasus_samsum",
    do_train=True,
    do_eval=True,
    logging_steps=100,
    save_steps=500,
    save_total_limit=2,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    learning_rate=5e-5,
    weight_decay=0.01,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
    logging_dir="./logs",
    report_to=[]  # disables W&B
)




In [ ]:
pip install --upgrade transformers accelerate datasets


In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from datasets import load_dataset
import torch

# Load dataset
dataset_samsum_pt = load_dataset("knkarthick/samsum")

# Load model and tokenizer
model_name = "google/pegasus-large"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

# Tokenize dataset
def preprocess_function(examples):
    model_inputs = tokenizer(examples["dialogue"], max_length=1024, truncation=True)
    labels = tokenizer(examples["summary"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset_samsum_pt.map(preprocess_function, batched=True)

# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Training arguments (disable wandb)
training_args = Seq2SeqTrainingArguments(
    output_dir="./pegasus_samsum",
    do_train=True,
    do_eval=True,
    logging_steps=100,
    save_steps=500,
    save_total_limit=2,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    learning_rate=5e-5,
    weight_decay=0.01,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
    logging_dir="./logs",
    report_to=[]  # disables W&B
)

# Initialize trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Train
trainer.train()


In [ ]:
trainer.train()        # for training
trainer.evaluate()     # for evaluation


In [ ]:
# Evaluate on the validation set
metrics = trainer.evaluate(tokenized_datasets["validation"])
print(metrics)


In [ ]:
from datasets import load_metric

rouge = load_metric("rouge")

# Take a small subset for quick evaluation
eval_samples = tokenized_datasets["validation"].select(range(20))

predictions = trainer.predict(eval_samples)
decoded_preds = tokenizer.batch_decode(predictions.predictions, skip_special_tokens=True)
decoded_labels = tokenizer.batch_decode(predictions.label_ids, skip_special_tokens=True)

# Compute ROUGE
results = rouge.compute(predictions=decoded_preds, references=decoded_labels)
print(results)
